# Fake Video Detection using eye blinking

## Importing liberaries

In [2]:
import numpy as np
import cv2
import dlib
from scipy.spatial import distance as dist
import os
notebook_path = os.path.abspath("Notebook.ipynb")

### Function to calculate eye aspect ratio

In [3]:
def eye_aspect_ratio(eye):
    # compute the euclidean distance between the vertical eye landmarks
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal eye landmarks
    C = dist.euclidean(eye[0], eye[3])

    # compute the EAR
    ear = (A + B) / (2 * C)
    return ear

### Eye aspect ratio

In [4]:
EYE_AR_THRESH = 0.22
EYE_AR_CONSEC_FRAMES = 3
EAR_AVG = 0

### initialize dlib's face detector (HOG-based) and then create

In [5]:
def count_blinks(video):
    # eye features regions
    RIGHT_EYE_POINTS = list(range(36, 42))
    LEFT_EYE_POINTS = list(range(42, 48))


    COUNTER = 0
    TOTAL = 0

    # to detect the facial region
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(os.path.join(os.path.dirname(notebook_path), "shape_predictor_68_face_landmarks.dat"
    ))
    # capture frame from local video
    # cap = cv2.VideoCapture("Home/Eye-detect-from-frame/download.jpeg")
    # capture video from live facecam
    cap = cv2.VideoCapture(0)

    cap = cv2.VideoCapture(os.path.join(os.path.dirname(notebook_path), video))

    while True:
        # get the frame
        ret, frame = cap.read()
        #frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        if frame is not []:
            # convert the frame to grayscale
            try:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            except:
                print(str(TOTAL)+" blinks found")
                return TOTAL
                cv2.destroyAllWindows()
                break;
            # get the facial regions
            rects = detector(gray, 0)
            # loop through each of the facial regions
            for rect in rects:
                x = rect.left()
                y = rect.top()
                x1 = rect.right()
                y1 = rect.bottom()
                # get the facial landmarks
                landmarks = np.matrix([[p.x, p.y] for p in predictor(frame, rect).parts()])
                # get the left eye landmarks
                left_eye = landmarks[LEFT_EYE_POINTS]
                # get the right eye landmarks
                right_eye = landmarks[RIGHT_EYE_POINTS]
                # draw contours on the eyes
                left_eye_hull = cv2.convexHull(left_eye)
                right_eye_hull = cv2.convexHull(right_eye)
                cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1) # (image, [contour], all_contours, color, thickness)
                cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)
                # compute the EAR for the left eye
                ear_left = eye_aspect_ratio(left_eye)
                # compute the EAR for the right eye
                ear_right = eye_aspect_ratio(right_eye)
                # compute the average EAR
                ear_avg = (ear_left + ear_right) / 2.0
                # detect the eye blink
                if ear_avg < EYE_AR_THRESH:
                    COUNTER += 1
                else:
                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        TOTAL += 1

                    COUNTER = 0

                cv2.putText(frame, "Blinks {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
                cv2.putText(frame, "EAR {}".format(ear_avg), (10, 60), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            cv2.imshow("Winks Found", frame)
            key = cv2.waitKey(1) & 0xFF
            # When key 'Q' is pressed, exit
            if key is ord('q'):
                print(TOTAL)
                break
        else:
            cv2.destroyAllWindows()
    # release all resources
    cv2.waitKey(0)
    # destroy all windows
    cv2.destroyAllWindows()




In [6]:
count_blinks("vid.mp4")

25 blinks found


25